## Agents in Teams

In this section we will learn how to build multi-agent teams.

Different Type of Teams Architecture:

#### **RoundRobinGroupChat()**
It is a simple team configuration where all the agents share the same context and take turns responding in a round robin fashion. Each agent during its turn broadcast its response to all other agents, ensuring that the entire team maintains the same context.


In [1]:
import os
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage

load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')
openai_model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

### Single Agent vs MultiAgent

In [2]:
single_agent = AssistantAgent(
    name='singlestoryagent',
    model_client=openai_model_client,
    system_message='You are a creative writer. Generate a short story about the plot user provides you.'
)

async def test_single_agent():
    
    task = TextMessage(content='Write a short story about a knight and a dog. Keep it in 300 words', source='user')
    result = await single_agent.run(task=task)
    
    print(result.messages[-1].content)
    
await test_single_agent()

In the kingdom of Eldoria, where mountains kissed the sky and forests whispered secrets, there lived a valiant knight named Sir Cedric. A loyal protector of the realm, he was known for his bravery and unwavering sense of justice. Yet, despite his accolades, his heart felt heavy and lonely. 

One misty morning, while patrolling the edge of the Blackwood Forest, he stumbled upon a scruffy, golden-furred dog. The creature had a torn ear and a cautious gaze, but there was an undeniable spark of courage in its eyes. Sir Cedric knelt and offered a morsel of bread, which the dog devoured with eager gratitude. 

“From this day forth, I shall name you Bristle,” he declared, and from that moment, they became inseparable. Together, they ventured beyond daunting mountains and treacherous rivers, facing foes and forging friendships alike. Bristle’s keen instincts often led Sir Cedric to danger, but also to glory, earning them both a legendary status throughout Eldoria.

One fateful night, dark clou

### Agents Team using RoundRobinGroupChat()

In [3]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

plot_agent = AssistantAgent(
    name='plot_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You create engaging plots for stories. Focus on creating the plot for the story based on the user prompt. You will only create the plot of the story'
)

character_agent = AssistantAgent(
    name= 'character_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You develop each character of the stories in detals, incluing their motivation and background based on the plot provided by "plot_agent". Make sure you only develop the characters and their background and motivation.'
)

editing_agent = AssistantAgent(
    name = 'editing_agent',
    model_client=openai_model_client,
    system_message='You are a creative writer. You will write the complete story based on the plot provided by "plot_agent" and characters and their backgroud provided by "character_agent"'
)

In [4]:
## agent team
agent_team = RoundRobinGroupChat(
    participants=[plot_agent,character_agent,editing_agent],
    ## max_turns 3 means in total there will be 3 runs, when a agent runs that is counted as 1 run.
    max_turns=3
)

async def test_team_output():
    
    task = TextMessage(content='Write a short story about a knight and a dog. Keep it in 300 words', source='user')
    result = await agent_team.run(task=task)
    
    for each_message in result.messages:
        print(f'{each_message.source} : {each_message.content}')
        print('\n\n')
        
await test_team_output()

user : Write a short story about a knight and a dog. Keep it in 300 words



plot_agent : In the medieval kingdom of Eldoria, Sir Cedric, a valiant knight known for his bravery, lived a simple life protecting the realm from dragons and marauders. However, it was not his sword skills that earned him the hearts of the people, but his unbreakable bond with Alaric, a scruffy, one-eyed dog he rescued from a pit on one of his quests.

One fateful day, Eldoria was threatened by the return of a fearsome dragon named Grimbold. It had laid waste to villages and demanded tribute, leaving fear in its wake. The king summoned all knights to confront the beast, but Sir Cedric, though brave, feared for Alaric’s safety if he left him behind.

Clad in shining armor, Cedric and Alaric embarked on the journey to confront Grimbold. They faced treacherous terrains and managed to outsmart traps laid by the dragon’s minions. With Alaric's keen senses and unwavering loyalty, they discovered that the dragon was

## Observing the Running Agent Team

In [21]:
from autogen_agentchat.base import TaskResult
total_prompt_token = 0
total_completion_token = 0
async for message in agent_team.run_stream(task='Write a poem on Love in 20 words'):
    if isinstance(message, TaskResult):
        print(f"Stop Reason : {message.stop_reason}")
        print(f"Total Tokens {total}")
    #print(message.content)
    elif message.source =='user':
        print(f"Source:{message.source}: \n{message.content}")
    else:
        print(f"Source:{message.source}: \n{message.content}")
        total_prompt_token += message.models_usage.prompt_tokens
        total_completion_token += message.models_usage.completion_tokens
        total = total_completion_token+total_prompt_token
        print(f"Prompt Token {message.models_usage.prompt_tokens}")
        print(f"Completion Token {message.models_usage.completion_tokens}")
        
    print("-"*60)

Source:user: 
Write a poem on Love in 20 words
------------------------------------------------------------
Source:plot_agent: 
In soft whispers, love takes flight,  
Hearts entwined, a dance of light.  
Together we dream, hand in hand,  
An eternal bond, forever grand.  
Prompt Token 6806
Completion Token 37
------------------------------------------------------------
Source:character_agent: 
In soft whispers, love takes flight,  
Hearts entwined, a dance of light.  
Together we dream, hand in hand,  
An eternal bond, forever grand.  
Prompt Token 6860
Completion Token 37
------------------------------------------------------------
Source:editing_agent: 
In soft whispers, love takes flight,  
Hearts entwined, a dance of light.  
Together we dream, hand in hand,  
An eternal bond, forever grand.  
Prompt Token 6893
Completion Token 37
------------------------------------------------------------
Stop Reason : Maximum number of turns 3 reached.
Total Tokens 20670
------------------------

In [27]:
await Console(
    agent_team.run_stream(task="Write a poem on Life in 20 words")
)

---------- TextMessage (user) ----------
Write a poem on Life in 20 words


---------- TextMessage (plot_agent) ----------
Life's a journey, twists and turns,  
With every lesson, a new hope burns.  
Moments fleeting, memories to keep,  
In laughter and love, our dreams run deep.  
---------- TextMessage (character_agent) ----------
Life's a journey, twists and turns,  
With every lesson, a new hope burns.  
Moments fleeting, memories to keep,  
In laughter and love, our dreams run deep.  
---------- TextMessage (editing_agent) ----------
Life's a journey, twists and turns,  
With every lesson, a new hope burns.  
Moments fleeting, memories to keep,  
In laughter and love, our dreams run deep.  


TaskResult(messages=[TextMessage(id='5129e6d2-3164-4106-b96a-ab32f54047b0', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 52, 52, 810337, tzinfo=datetime.timezone.utc), content='Write a poem on Life in 20 words', type='TextMessage'), TextMessage(id='0116f387-c3c7-491b-a6e5-ba181eb1b57b', source='plot_agent', models_usage=RequestUsage(prompt_tokens=7857, completion_tokens=40), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 52, 54, 48364, tzinfo=datetime.timezone.utc), content="Life's a journey, twists and turns,  \nWith every lesson, a new hope burns.  \nMoments fleeting, memories to keep,  \nIn laughter and love, our dreams run deep.  ", type='TextMessage'), TextMessage(id='afe4e925-cace-4116-9095-2448733bdc7c', source='character_agent', models_usage=RequestUsage(prompt_tokens=7914, completion_tokens=40), metadata={}, created_at=datetime.datetime(2025, 7, 25, 6, 52, 55, 627350, tzinfo=datetime.timezone.utc), content="Life's a jou